<a href="https://colab.research.google.com/github/HungHoangDinh/MFEAs/blob/main/MFEAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

In [2]:
population_size=50
dimension=5
shift=[random.uniform(-1, 1) for _ in range(dimension)]

In [3]:
def ackley_function(x,shift):
    n = len(x)
    sum_sq_term = -0.2 * np.sqrt((1/n) * np.sum((x - shift) ** 2))
    cos_term = np.sum(np.cos(2 * np.pi * (x - shift)))
    return -20 * np.exp(sum_sq_term) - np.exp((1/n) * cos_term) + 20 + np.exp(1)

def sphere_function(x):
    return np.sum(x**2)

In [4]:
def initialize_population(population_size, dimension):
    return np.random.uniform(-5.12, 5.12, size=(population_size, dimension))

In [5]:
population=initialize_population(population_size,dimension)

In [6]:
def cost_ackley(population):
    result = []
    for x in population:
        result.append(ackley_function(x,shift));
    return np.array(result)
def cost_sphere(population):
    result = []
    for x in population:
        result.append(sphere_function(x));
    return np.array(result)

In [7]:
def rank_ackley(population):
    return np.argsort(np.argsort(cost_ackley(population)))
def rank_sphere(population):
    return np.argsort(np.argsort(cost_sphere(population)))

In [8]:
def evalution(population):
  rank_1=rank_ackley(population)
  rank_2=rank_sphere(population)
  skill_factor=[];
  for i in range(population_size):
        if rank_1[i] < rank_2[i]:
            skill_factor.append(0)
        else:
            skill_factor.append(1)
  return np.array(skill_factor)

In [9]:
def crossover(parent1, parent2):
    crossover_point = np.random.rand()
    child1 = parent1 * crossover_point + parent2 * (1 - crossover_point)
    child2 = parent2 * crossover_point + parent1 * (1 - crossover_point)
    return child1, child2

In [10]:
def mutation(individual, mutation_rate, mutation_scale):
    mutated_individual = individual.copy()
    for i in range(len(mutated_individual)):
        if np.random.rand() < mutation_rate:
            mutated_individual[i] += np.random.normal(scale=mutation_scale)
    return mutated_individual


In [21]:

def assortative_mating(cur_pop, skill_factor):
    offsprings = []
    offsprings_skill_factor = []
    rmp=0.3
    for i in range(population_size):
        rnd = np.random.rand(1)[0]
        random_indices = np.random.choice(np.arange(len(cur_pop)), size=2, replace=False)
        if skill_factor[random_indices[0]] == skill_factor[random_indices[1]] or rnd < rmp:
            offspring1,offspring2=crossover(cur_pop[random_indices[0]],cur_pop[random_indices[1]]);
            offsprings.append(offspring1)
            random_skill_factor = np.random.rand(1)[0]
            if random_skill_factor < 0.5:
                offsprings_skill_factor.append(skill_factor[random_indices[0]])
            else:
                offsprings_skill_factor.append(skill_factor[random_indices[1]])
        else:
            random_numbers = np.random.choice(np.arange(0, dimension), size=2, replace=False)
            offspring0 = cur_pop[random_indices[0]].copy()
            offspring1 = cur_pop[random_indices[1]].copy()
            offspring0=mutation(offspring0,0.1,0.1);
            offspring1=mutation(offspring1,0.1,0.1);
            offsprings.append(offspring0)
            offsprings_skill_factor.append(skill_factor[random_indices[0]])
            offsprings.append(offspring1)
            offsprings_skill_factor.append(skill_factor[random_indices[1]])

    return np.array(offsprings), np.array(offsprings_skill_factor)

In [12]:
def update_scalar_fitness(population, s):
    ackley_pop = population[np.where(s == 0)[0]]
    rank1 = rank_ackley(ackley_pop)
    scalar_fitness_ackley = [1/(i+1) for i in rank1]

    sphere_pop = population[np.where(s == 1)[0]]
    rank2 = rank_sphere(sphere_pop)
    scalar_fitness_sphere = [1/(i+1) for i in rank2]
    scalar_fitness = np.random.rand(population.shape[0])
    scalar_fitness[np.where(s == 0)[0]] = scalar_fitness_ackley
    scalar_fitness[np.where(s == 1)[0]] = scalar_fitness_sphere
    filter_indices = np.argsort(scalar_fitness)[:10]
    fittest_pop = np.delete(population, filter_indices, axis=0)
    fittest_skill_factor = np.delete(s, filter_indices)
    return fittest_pop, fittest_skill_factor


In [13]:
def main():
    population=initialize_population(population_size,dimension)
    skill_factor=evalution(population)
    generation = 400
    for i in range(generation):
        offspring_pop, offspring_skill_factor = assortative_mating(population, skill_factor)
        intermediate_pop = np.vstack((population, offspring_pop))
        intermediate_skill_factor = np.concatenate((skill_factor, offspring_skill_factor))
        population, skill_factor = update_scalar_fitness(intermediate_pop, intermediate_skill_factor)
        ackley_pop = population[np.where(skill_factor == 0)[0]]
        sphere_pop = population[np.where(skill_factor == 1)[0]]
    print("Ackley: ", cost_ackley(ackley_pop).min())
    print("Sphere: ", cost_sphere(sphere_pop).min())

In [22]:
main();

Ackley:  1.109297069773572
Sphere:  0.22425124868264476
